# New Hampshire Education Tax Credit
New Hampshire's Education Tax Credit provides up to $510_000, depending on the donation amount.

In [24]:
from policyengine_us import Simulation
from policyengine_us import IndividualSim
import plotly.express as px
import pandas as pd


LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}

def make_education_tax_credit(): 
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="NH")
    sim.vary("charitable_cash_donations", max=800_000, step=1_000) #? How to include "charitable_non_cash_donations" in vary function?
    return pd.DataFrame(
        dict(
            charitable_cash_donations=sim.calc("charitable_cash_donations")[0],
            nh_education_tax_credit=sim.calc("nh_education_tax_credit")[0].round(),
            mtr=sim.deriv(
                "nh_education_tax_credit", "charitable_cash_donations", wrt_target="head"
            ),
        )
        )

l = []
l.append(make_education_tax_credit())

df = pd.concat(l)

LABELS = dict(
    charitable_cash_donations="Donations",
    nh_education_tax_credit="New Hampshire education tax credit",
    mtr="Marginal tax rate",
)

fig = px.line(
    df,
    "charitable_cash_donations",
    "nh_education_tax_credit",
    #animation_frame="year",
    labels=LABELS,
    title="New Hampshire education tax credit",
    color_discrete_map=COLOR_MAP,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()